In [1]:
# Calculate the intrinsic value
import glob
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

ticker = 'INTC'

In [2]:
# Get the latest stored data - annual reports
annuals = glob.glob(f'./annual_reports/{ticker}_*_annual_*.csv')
assert len(annuals) > 0, f'No annual files found for {ticker}'
annuals.sort()
df_annuals = pd.read_csv(annuals[-1], sep=';', header=0, index_col='fiscalYear')

In [3]:
# Get the overview data
overviews = glob.glob(f'./overviews/{ticker}_*_overview_*.csv')
assert len(overviews) > 0, f'No overview files found for {ticker}'
overviews.sort()
df_overviews = pd.read_csv(overviews[-1], sep=';', header=0, index_col='Metric')

In [4]:
# def calculate_tax_rate (df):
#     return (df_annuals.loc[df_annuals.index.max(), 'incomeTaxExpense'] / df_annuals.loc[df_annuals.index.max(), 'incomeBeforeTax']) * 100

def calculate_tax_rate(income_tax_expense, income_bef_tax):
    return (income_tax_expense / income_bef_tax)

def calculate_total_debt_ratio(debt, market_cap):
    return debt / (debt + market_cap)

def calculate_market_cap_ratio(debt, market_cap):
    return market_cap / (debt + market_cap)

def calculate_cost_of_debt(interest_expense, total_debt):
    return interest_expense / total_debt

def calculate_cost_of_debt_after_tax(cost_of_debt, tax_rate):
    return cost_of_debt * (1 - tax_rate)

def calculate_cost_of_equity(risk_free_rate, equity_risk_premium, beta):
    return risk_free_rate + (equity_risk_premium * beta)

def calculate_WACC(cost_of_equity, cost_of_debt_after_tax, market_cap_ratio, total_debt_ratio):
    '''Weighted average cost of capital'''
    return (cost_of_equity * market_cap_ratio) + (cost_of_debt_after_tax * total_debt_ratio)

def xnpv(rate, values, dates):
    '''Equivalent of Excel's XNPV function.
       Based on: https://stackoverflow.com/a/33260133
    >>> from datetime import date
    >>> dates = [date(2010, 12, 29), date(2012, 1, 25), date(2012, 3, 8)]
    >>> values = [-10000, 20, 10100]
    >>> xnpv(0.1, values, dates)
    -966.4345...
    '''
    if rate <= -1.0:
        return float('inf')
    d0 = dates[0]    # or min(dates)
    return sum([ vi / (1.0 + rate)**((di - d0).days / 365.0) for vi, di in zip(values, dates)])
    
def calc_growth_rate(val_start, val_end):
    return (val_end / val_start) - 1

def get_avg_growth_values(n_periods, growth_rate, val_start):
    out_vals = []
    for period in range(n_periods):
        if period == 0:
            val = val_start
        val = val + (val * growth_rate)
        out_vals.append(val)
    return out_vals

In [5]:
# Assumptions

# General

# The reasonable rate (%) that cash flow can grow
perpetual_growth_rate = 0.025
# US gov bond 10y yield
risk_free_rate = 0.013
equity_risk_premium = 0.075

# Company-specific


# discount_rate = calculate_WACC
EV_EBITDA = df_overviews.loc['EVToEBITDA'].astype('float')[0]
# current_price = 
shares_outstanding = df_overviews.loc['SharesOutstanding'].astype('int64')[0]
market_capitalization = int(df_overviews.loc['MarketCapitalization', 'Value'])
total_debt = df_annuals.loc[df_annuals.index.max(), ['shortTermDebt', 'longTermDebt']].sum()
cash = df_annuals.loc[df_annuals.index.max(), ['cashAndShortTermInvestments']].astype('float')[0]
capex = df_annuals.loc[:, ['capitalExpenditures']].mean()[0]
beta = df_overviews.loc['Beta',:].astype('float')[0]

In [6]:
# WACC Calculations
tax_rate = calculate_tax_rate(abs(df_annuals.loc[df_annuals.index.max(), 'incomeTaxExpense']),
                              df_annuals.loc[df_annuals.index.max(), 'incomeBeforeTax'])
market_cap_r = calculate_market_cap_ratio(total_debt, market_capitalization)
total_debt_r = calculate_total_debt_ratio(total_debt, market_capitalization)
cost_of_equity = calculate_cost_of_equity(risk_free_rate, equity_risk_premium, beta)
cost_of_debt = calculate_cost_of_debt(interest_expense=abs(df_annuals.loc[df_annuals.index.max(), 'interestExpense']),
                                      total_debt=total_debt)
cost_of_debt_after_tax = calculate_cost_of_debt_after_tax(cost_of_debt, tax_rate)


discount_rate = calculate_WACC(cost_of_equity,
                               cost_of_debt_after_tax,
                               market_cap_r,
                               total_debt_r)
print(f'Discount rate: {round(discount_rate*100, 2)}%')

Discount rate: 5.7%


In [7]:
calc_growth_rate(38741190267, 45650857619)

0.17835454472047285

In [8]:
# EBIT avg growth rate for the past 4 years
ebit_avg_growth = np.mean(df_annuals.loc[:, 'ebit'].sort_index().pct_change().dropna().to_list())
future_ebit_vals = pd.Series(get_avg_growth_values(4, ebit_avg_growth, df_annuals.loc[df_annuals.index.max(), 'ebit']))

In [9]:
future_cash_taxes = future_ebit_vals * tax_rate
DA_avg = np.mean(df_annuals.loc[:, 'depreciationAndAmortization'].abs())
future_NWC = np.mean(pd.Series(df_annuals.loc[:, 'totalCurrentAssets']) - pd.Series(df_annuals.loc[:, 'totalCurrentLiabilities']))

In [30]:
# Next five years
next_5Y = list(range(int(df_annuals.index.max()) + 1, df_annuals.index.max() + 5)) 

df_dcf = future_ebit_vals.to_frame(name='EBIT')
df_dcf.index = next_5Y
df_dcf['Cash taxes'] = df_dcf.loc[:, 'EBIT'] * tax_rate
df_dcf['Depr&Amort'] = np.mean(df_annuals.loc[:, 'depreciationAndAmortization'].abs())
df_dcf['Capex'] = capex
# This needs to be change in NWC, not NWC!
df_dcf['NetWorkCap'] = np.mean((pd.Series(df_annuals.loc[:, 'totalCurrentAssets']) - pd.Series(df_annuals.loc[:, 'totalCurrentLiabilities'])).diff().dropna())
df_dcf['unleveredFCF'] = (df_dcf['EBIT'] - df_dcf['Cash taxes'] + df_dcf['Depr&Amort'] - df_dcf['Capex'] - df_dcf['NetWorkCap'])

In [31]:
np.mean((pd.Series(df_annuals.loc[:, 'totalCurrentAssets']) - pd.Series(df_annuals.loc[:, 'totalCurrentLiabilities'])).diff().dropna())

1822250000.0

In [37]:
df_dcf['unleveredFCF']

2021    8.789791e+09
2022    1.293683e+10
2023    1.782351e+10
2024    2.358174e+10
Name: unleveredFCF, dtype: float64

2021    2.790108e+10
2022    3.287736e+10
2023    3.874119e+10
2024    4.565086e+10
Name: EBIT, dtype: float64

In [10]:
df_overviews

,Value
Metric,
Symbol,INTC
AssetType,Common Stock
Name,Intel Corporation
Description,"Intel Corporation designs, manufactures, and s..."
Exchange,NASDAQ
Currency,USD
Country,USA
Sector,Technology
Industry,Semiconductors
